#  Azure ML and IoT Edge


Ensure we have a consistent version of the Azure ML SDK.

In [ ]:
import sys
! {sys.executable} -m pip install -q --upgrade azureml-sdk[notebooks,automl,contrib]==1.5.0

In [ ]:
from azureml.core.model import Model
from azureml.core.environment import Environment

import warnings
warnings.filterwarnings('ignore')

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [ ]:
# Check core SDK version number
import azureml.core
from azureml.core import Workspace

print("SDK version:", azureml.core.VERSION)

## 1: Specify parameters
Fill in the parameters below.  If you already have IoT Hub or Azure ML workspace, then enter their information here. Otherwise, the parameter names will be used in provisioning new services.

In [ ]:
# Provide the same experiment suffix used in the PyTorch training notebook. Replace ***
my_nickname = ***

# Provide your Azure subscription ID to provision your services
subscription_id = ""

# Provide your Azure ML service resource group and workspace name 
# If you don't have a workspace, pick a name to create a new one
resource_group_name_aml = ""
aml_workspace_name = ""

In [ ]:
# DO NOT CHANGE THESE VALUES for this tutorial

# Enter the resource group in Azure where you want to provision the resources
# or where IoT Hub exists
resource_group_name_iot = "iot-aicamp-" + my_nickname

# Enter Azure region where your IoT services will be provisioned, for example "eastus2"
azure_region = "eastus2"

# Enter your Azure IoT Hub name 
# If you don't have an IoT Hub, pick a name to make a new one 
iot_hub_name = "iothub-aicamp-" + my_nickname

# Enter your IoT Edge device ID 
# If you don't have an IoT Edge device registered, pick a name to create a new one 
# This is NOT the name of your VM, but it's just an entry in your IoT Hub, so you can pick any name
iot_device_id = "edge-vm-device"

# Enter a name for the IoT Edge VM
edge_vm_name = "edge-vm-" + my_nickname

# This is the name of the AML module you deploy to the device
module_name = "machinelearningmodule"

The login command below will trigger interactive login.  Follow the directions printed below.

In [ ]:
! sudo az login

In [ ]:
# Just in case this is the command to update the Azure CLI
# ! curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash

In [ ]:
# Load the IoT extension for Azure CLI if needed - may not work in the notebook so try in terminal if not
! sudo az extension add --name azure-cli-iot-ext

In [ ]:
! sudo az account set --subscription $subscription_id

In [ ]:
! sudo az group create --name $resource_group_name_iot --location $azure_region

## 2: Provision IoT Hub 
If you already have provisioned these resources, then skip this section and go Section 3.

### 2.1 Provision an Edge VM

**IMPORTANT NOTE**: Before you proceed, you must perform a one-time task to accept the terms of the data science virtual machine on your Azure subscription. You can do this by visiting [Configure Programmatic Deployment](https://ms.portal.azure.com/#blade/Microsoft_Azure_Marketplace/LegalTermsSkuProgrammaticAccessBlade/legalTermsSkuProgrammaticAccessData/%7B%22product%22%3A%7B%22publisherId%22%3A%22microsoft_iot_edge%22%2C%22offerId%22%3A%22iot_edge_vm_ubuntu%22%2C%22planId%22%3A%22ubuntu_1604_edgeruntimeonly%22%7D%7D)

In [ ]:
! sudo az vm create --resource-group $resource_group_name_iot --name $edge_vm_name --image microsoft_iot_edge:iot_edge_vm_ubuntu:ubuntu_1604_edgeruntimeonly:latest --admin-username azureuser --generate-ssh-keys


            

If you want to SSH into this VM after setup, use the publicIpAddress with the command: `ssh azureuser@{publicIpAddress}`.  To open up ports for SSH issues see <a href="https://docs.microsoft.com/en-us/azure/iot-edge/how-to-install-iot-edge-ubuntuvm#next-steps" target="_blank">this resource</a>.

### 2.2: Provision IoT Hub

If you get an error because there's already one free hub in your subscription, change the SKU to S1. If you get an error that the IoT Hub name isn't available, it means that someone else already has a hub with that name so try a different name.


In [ ]:
! sudo az iot hub create --resource-group $resource_group_name_iot --name $iot_hub_name --sku F1

### 2.3 Register an IoT Edge device

In [ ]:
# Register an IoT Edge device (create a new entry in the Iot Hub)
! sudo az iot hub device-identity create --hub-name  $iot_hub_name --device-id $iot_device_id --edge-enabled

## 3: Load resources
Load the Azure ML workspace and get the IoT Edge device connection string from your IoT Hub.

### 3.1 Load the Azure ML workspace.

In [ ]:
# Initialize a workspace object from persisted configuration
from azureml.core import Workspace

ws = Workspace.from_config(path="config.json")

print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

### 3.2: Get the Azure IoT Edge device connection string

Set the Edge connection string on the device.

Instruction can be found here:  https://docs.microsoft.com/en-us/azure/iot-edge/how-to-install-iot-edge-linux#configure-the-security-daemon.

In [ ]:
# Get the connection string that you will need to enter in the IoT Edge device
! sudo az iot hub device-identity show-connection-string --device-id $iot_device_id --hub-name $iot_hub_name

Replace the `HostName=...` in the following variable with the entire connection string from above.

In [ ]:
# Secret!!!  Don't check in to source control
conn_str = "HostName=..."

# Don't modify this part
set_cmd = "/etc/iotedge/configedge.sh '"+conn_str+"'"
print(set_cmd)

In [ ]:
! sudo az vm run-command invoke -g $resource_group_name_iot -n $edge_vm_name --command-id RunShellScript --script "$set_cmd"

## 4: PyTorch Classification Model

We've already:
- Trained the model
- Created the scoring script
- Deployed it as a service to Azure Container Instance

### 4.1 Get registered model

In [ ]:
model = Model(ws,'behavior-pytorch-'+my_nickname, version=1)

### 4.2 Create Docker Image

Specify the required packages for the image.

In [ ]:
# This specifies the dependencies to include in the environment
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(pip_packages=['azureml-defaults==1.5.0', 
                                               'torch==1.3.0', 
                                               'torchvision==0.4.1',
                                               'Pillow==6.2.1'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())
    
print(myenv.serialize_to_string())

You can add tags and descriptions to images. Also, an image can contain multiple models.

In [ ]:
from azureml.core.image import Image, ContainerImage

image_config = ContainerImage.image_configuration(runtime="python",
                                 execution_script="pytorch_score_iot.py",
                                 conda_file="myenv.yml",
                                 tags={'area': "iot", 'type': "classification", "framework": "pytorch"},
                                 description="IoT Edge PyTorch classification model for suspicious behavior; Pillow<7")


image = Image.create(name="suspiciousbehaviorclass",
                     # this is the model object 
                     models=[model],
                     image_config=image_config, 
                     workspace=ws)

Note that following command can take few minutes. 

In [ ]:
image.wait_for_creation(show_output=True)

List images by tag and find out the detailed build log for debugging.

In [ ]:
for i in Image.list(workspace=ws, tags=["area"]):
    print('{}(v.{} [{}]) stored at {} with build log {}'.format(i.name, i.version, i.creation_state, i.image_location, i.image_build_log_uri))

## 5: Deploy container to Azure IoT Edge device
Create a deployment.json file that contains the modules you want to deploy to the device and the routes.  Then push this file to the IoT Hub, which will then send it to the IoT Edge device.  The IoT Edge agent will then pull the Docker images and run them.

In [ ]:
# Getting your container details
container_reg = ws.get_details()["containerRegistry"]
reg_name=container_reg.split("/")[-1]
container_url = "\"" + image.image_location + "\","
subscription_id = ws.subscription_id
print('{}'.format(image.image_location))
print('{}'.format(reg_name))
print('{}'.format(subscription_id))
from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azure.mgmt import containerregistry
client = ContainerRegistryManagementClient(ws._auth,subscription_id)
result= client.registries.list_credentials(resource_group_name_aml, reg_name, custom_headers=None, raw=False)
username = result.username
password = result.passwords[0].value

The file modified below is a standar IoT Edge manifest file.  This is how IoT Edge and IoT Hub know what modules to deploy down to the device (which in this case is an Azure VM running IoT Edge Runtime).

In [ ]:
file = open('iot-deployment-template.json')
contents = file.read()
contents = contents.replace('__MODULE_NAME', module_name)
contents = contents.replace('__REGISTRY_NAME', reg_name)
contents = contents.replace('__REGISTRY_USER_NAME', username)
contents = contents.replace('__REGISTRY_PASSWORD', password)
contents = contents.replace('__REGISTRY_IMAGE_LOCATION', image.image_location)
with open('./deployment.json', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)

The following commmand will tell IoT Hub to deploy down the modules from images in the ACR.

In [ ]:
# Push the deployment JSON to the IOT Hub
! sudo az iot edge set-modules --device-id $iot_device_id --hub-name $iot_hub_name --content deployment.json

## Congratulations!
You made it to the end of the tutorial!  You can monitor messages from your edge device to your IoT Hub with VS Code and the [Azure IoT Hub Toolkit](https://marketplace.visualstudio.com/items?itemName=vsciot-vscode.azure-iot-toolkit) extension.

After installing the extension in VSCode, log in to the Azure Account (View -> Command Palette -> "Azure: Sign in to Azure Cloud").

Select the IoT Hub by:  View -> Command Palette -> "Azure IoT Hub:  Set IoT Hub Connection String".

<img width="50%" src="../../assets/iot_edge_select_iot_hub_conn_str.png">

Monitor the built in endpoint by right clicking on the Device under the Azure IoT Hub (expand this in the lower left corner), selecting "Start Monitoring Built-in Event Endpoint" (will monitor all messages from any device module to IoT Hub).

<img width="50%" src="../../assets/iot_edge_monitor_vscode.png">

If selecting "Start Monitoring Built-in Event Endpoint" in VSCode, the output should look like:


```

[IoTHubMonitor] Start monitoring message arrived in built-in endpoint for device [edge-vm-device] ...
[IoTHubMonitor] Created partition receiver [0] for consumerGroup [$Default]
[IoTHubMonitor] Created partition receiver [1] for consumerGroup [$Default]
[IoTHubMonitor] [7:59:45 PM] Message received from [edge-vm-device/machinelearningmodule]:
{
  "body": {
    "label": "suspicious",
    "probability": "0.5001148",
    "filename": "Walk3frame0002.jpg"
  },
  "applicationProperties": {
    "AzureMLResponse": "OK"
  }
}
[IoTHubMonitor] [7:59:45 PM] Message received from [edge-vm-device/machinelearningmodule]:
{
  "body": {
    "label": "suspicious",
    "probability": "0.5",
    "filename": "Browse_WhileWaiting2frame0000.jpg"
  },
  "applicationProperties": {
    "AzureMLResponse": "OK"
  }
}
```